In [ ]:
import os
iskaggle = os.environ.get('KAGGLE_KERNAL_RUN_TYPE', '')

if iskaggle:
  !pip install -Uqq fastai

In [ ]:
from fastcore.all import *
import time
import json

def search_images(term, max_images=100):
  url = 'https://duckduckgo.com/'
  res = urlread(url,data={'q':term})
  time.sleep(2)
  searchObj = re.search(r'vqd=([\d-]+)\&', res)
  requestUrl = url + 'i.js'
  params = dict(l='us-en', o='json', q=term, vqd=searchObj.group(1), f=',,,', p='1', v7exp='a')    
  headers = dict( referer='https://duckduckgo.com/' )
  urls,data = set(),{'next':1}
  while len(urls)<max_images and 'next' in data:
      res = urlread(requestUrl, data=params, headers=headers)
      data = json.loads(res) if res else {}
      urls.update(L(data['results']).itemgot('image'))
      requestUrl = url + data['next']
      time.sleep(4)
  return L(urls)[:max_images]

In [ ]:
urls = search_images('blue oyster mushroom', max_images=1)
urls[0]

In [ ]:
from fastdownload import download_url
mushroom_types = 'blue oyster mushroom', 'shiitake mushroom', 'lion\'s mane mushroom', 'amanita muscaria mushroom'
path = Path('mushrooms')

for o in mushroom_types:
  dest = (path/o)
  dest.mkdir(exist_ok=True, parents=True)
  download_images(dest, urls=search_images(f'{o} photo'))
  time.sleep(5)
  resize_images(path/o, max_size=400, dest=path/o)

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)